# Basic Stock Analysis

How to evaluate the performance of your assets?
Optimize your portfolio by reviewing the statistics of your stocks.

### 1. Importing Libraries

In [2]:
import pandas_datareader as pdr
import numpy as np
import datetime as dt
import pandas as pd
from sklearn.linear_model import LinearRegression

### 2. Calculating Beta
Beta expresses how "dynamic" an asset reacts to market fluctuations. A high beta means high risk, but also high opportunity, depending on the market.
- Beta < 1: More stable, but also less growth opportunity
- Beta > 1: More risky, but also more gain opportunity

#### Method
- Get raw stock data of the market and the stocks of interest (can be 1 year, 3 years or other timeframe). Typically the S&P500 (Symbol "^GSPC") is used as the representation of the market data.
- Transform raw data into a dataframe with returns
- Calculate Beta
- Calculate the expected rate of return (CAPM method)
- Interprete the results

In [10]:
def calc_return(df):
    '''Function to generate a detaframe with returns
    - Input: Dataframe with raw stock data
    - Output: Dataframe with returns'''

    x = df.copy()
    
    for i in range(1, len(df)-1):
        x.iloc[i] = (df.iloc[i] - df.iloc[i-1]) / df.iloc[i-1] * 100

    x.iloc[0] = 0

    return x

In [11]:
tickers = ['XIC.TO', '^GSPC', 'SHOP', 'TSLA', 'VFV.TO']
start = dt.datetime(2018, 12, 1)
end = dt.datetime(2021, 12, 1)
data = pdr.get_data_yahoo(tickers, start, end, interval="d")

data = data['Adj Close']

log_returns = calc_return(data)

# Alternative way with similar results:
# log_returns = np.log(data / data.shift())

In [12]:
data

Symbols,XIC.TO,^GSPC,SHOP,TSLA,VFV.TO
Date,,,,,
2018-12-03,22.188370,2790.370117,156.389999,71.697998,62.759407
2018-12-04,21.905540,2700.060059,150.199997,71.940002,60.894699
2018-12-05,22.078886,NaN,NaN,NaN,62.176094
2018-12-06,21.732193,2695.949951,156.899994,72.612000,61.382401
2018-12-07,21.549723,2633.080078,146.880005,71.594002,59.737629
...,...,...,...,...,...
2021-11-25,34.549999,NaN,NaN,NaN,106.129997
2021-11-26,33.689999,4594.620117,1576.699951,1081.920044,104.459999
2021-11-29,33.730000,4655.270020,1567.020020,1136.989990,105.470001


In [13]:
log_returns

Symbols,XIC.TO,^GSPC,SHOP,TSLA,VFV.TO
Date,,,,,
2018-12-03,0.000000,0.000000,0.000000,0.000000,0.000000
2018-12-04,-1.274674,-3.236490,-3.958055,0.337533,-2.971201
2018-12-05,0.791332,NaN,NaN,NaN,2.104280
2018-12-06,-1.570247,NaN,NaN,NaN,-1.276525
2018-12-07,-0.839631,-2.332012,-6.386227,-1.401969,-2.679549
...,...,...,...,...,...
2021-11-25,0.611529,NaN,NaN,NaN,0.330874
2021-11-26,-2.489148,NaN,NaN,NaN,-1.573540
2021-11-29,0.118732,1.320020,-0.613936,5.090020,0.966879


### Calculating Beta

In [14]:
cov = log_returns.cov()
var = log_returns['^GSPC'].var()
beta_xic = cov.loc['XIC.TO', '^GSPC'] / var
beta_shop = cov.loc['SHOP', '^GSPC'] / var
beta_tsla = cov.loc['TSLA', '^GSPC'] / var
beta_vfv = cov.loc['VFV.TO', '^GSPC'] / var

In [15]:
print('Betas for XIC: {:1.2}, SHOPIFY: {:1.2}, Tesla: {:1.2}, Vanguard: {:1.2}'.format(beta_xic, beta_shop, beta_tsla, beta_vfv))

Betas for XIC: 0.0076, SHOPIFY: 0.32, Tesla: 0.24, Vanguard: 0.024
